In [1]:
# start to let c_EC affect j_SEI - start from solvent diffusion limited
# how?  
# 1. change: elif self.options["SEI"] == "solvent-diffusion limited": in sei_growth.py
# 2. change: self.C_sei_solvent in lithium_ion_parameters.py
# 4. now add calendar ageing- CV hold; for no CV hold, just set Para_0["Current function [A]"] = 0

In [2]:
import pybamm as pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os; #import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [3]:
years = 1
days = years * 365
hours = days * 24
print(hours)

8760


In [4]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.700000000000002e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Hold at {V_max} V for 1e4 hours (10 hour period)")    ] * 1 )  

In [5]:
Model_0 = pybamm.lithium_ion.DFN(     
    options={
        "SEI":"interstitial-diffusion limited",          
        "SEI film resistance":"distributed",          
        "SEI porosity change":"true",               
        "solvent diffusion": "EC"     } )
Model_1 = pybamm.lithium_ion.DFN(     
    options={      
        "SEI":"interstitial-diffusion limited",          
        "SEI film resistance":"distributed",          
        "SEI porosity change":"true",   
        "solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);

Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] =   "[input]";
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   "[input]";
Para_0['EC transference number'] =   "[input]";
Para_0['EC diffusivity [m2.s-1]'] =   "[input]";
Para_0['Electrolyte diffusivity [m2.s-1]'] =   "[input]";

Para_0.search("diffusivity")
gamma_e_ec_Rio = Para_0.evaluate(Model_0.param.gamma_e_ec_Rio)  
print(gamma_e_ec_Rio)

In [6]:
Sim_0    = pybamm.Simulation(
    Model_0, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)       
Sol_0    = Sim_0.solve(
    inputs={
        "EC transference number":-1.4,
        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "Typical EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "EC diffusivity in electrolyte [m2.s-1]":5E-12,
        "Electrolyte diffusivity [m2.s-1]":3E-10,
        });
Sim_1    = pybamm.Simulation(
    Model_1, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)       
""" Sol_1    = Sim_1.solve(
    inputs={
        "EC transference number":-1.4,
        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "Typical EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,
        "EC diffusivity in electrolyte [m2.s-1]":5E-12,
        "Electrolyte diffusivity [m2.s-1]":3E-10,
        }); """

' Sol_1    = Sim_1.solve(\n    inputs={\n        "EC transference number":-1.4,\n        "EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,\n        "Typical EC Lithium ion cross diffusivity [m2.s-1]":1.5e-9/1e2,\n        "EC diffusivity in electrolyte [m2.s-1]":5E-12,\n        "Electrolyte diffusivity [m2.s-1]":3E-10,\n        }); '

In [7]:
Model_0.variables.search('current density')

Current collector current density
Current collector current density [A.m-2]
Current density variable
Electrode current density
Electrolyte current density
Electrolyte current density [A.m-2]
Exchange current density
Exchange current density [A.m-2]
Inner SEI interfacial current density
Inner SEI interfacial current density [A.m-2]
Inner SEI on cracks interfacial current density
Inner SEI on cracks interfacial current density [A.m-2]
Interfacial current density
Interfacial current density [A.m-2]
Leading-order current collector current density
Lithium plating interfacial current density
Lithium plating interfacial current density [A.m-2]
Negative electrode SEI interfacial current density
Negative electrode SEI interfacial current density [A.m-2]
Negative electrode SEI on cracks interfacial current density
Negative electrode SEI on cracks interfacial current density [A.m-2]
Negative electrode SEI on cracks volumetric interfacial current density
Negative electrode SEI on cracks volumetric

In [11]:
label = ["SEI + double diffusion"] #,"SEI + single diffusion",
output_variables3 = [
    "Terminal voltage [V]",  
    "Current [A]", 
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "Total EC in electrolyte [mol]",
    "Loss of capacity to SEI [A.h]",
]
quick_plot = pybamm.QuickPlot([Sol_0,], output_variables3,label,) #     variable_limits='tight'Sol_1
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=10000.0, step=100.0), Output()), _dom_classe…

In [9]:
label = ["SEI + double diffusion"]  # ,"SEI + single diffusion",
output_variables3 = [
    "Terminal voltage [V]",   
    "Current [A]",
]
quick_plot = pybamm.QuickPlot([Sol_0], output_variables3,label,) #     variable_limits='tight',Sol_1
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=10000.0, step=100.0), Output()), _dom_classe…